In [1]:
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate
from surprise import accuracy
import pickle
import numpy as np
import data
import json

In [2]:
def calculate_metrics(measures):
    filename = "metrics/item_based.txt"
    with open(filename, "w+") as file:
        file.write(json.dumps(measures))
    print(f"saved metrics to {filename}")


def knn_train(trainset):
    sim_options = {
        "name": "pearson_baseline",
        "user_based": False,  # Compute similarities between items
    }
    model = KNNWithMeans(sim_options=sim_options)
    measures = cross_validate(model, trainset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

    # trainset = trainset.build_full_trainset()
    model.fit(trainset)
    return model, measures


In [3]:
df = data.load_ratings()
df = data.normalize_ratings(df)

In [4]:
df

,userId,movieId,rating
0,1,110,0.111111
1,1,147,0.888889
2,1,858,1.000000
3,1,1221,1.000000
4,1,1246,1.000000
...,...,...,...
26024284,270896,58559,1.000000
26024285,270896,60069,1.000000
26024286,270896,63082,0.888889
26024287,270896,64957,0.888889


In [ ]:
trainset = data.get_user_item_dataset(df)
model, measures = knn_train(trainset)
sim_options = {
        "name": "pearson_baseline",
        "user_based": False,  # Compute similarities between items
    }
model = KNNWithMeans(sim_options=sim_options)

try:
    measures = cross_validate(model, trainset, measures=['RMSE', 'MAE'], cv=5, verbose=True)
except Exception as e:
    print(e)

Estimating biases using als...


In [ ]:
trainset = trainset.build_full_trainset()
model.fit(trainset)

In [ ]:
with open('model/item_based_filtering.pkl', 'wb') as f:
    pickle.dump(model, f)
np.save("matrices/item_sim_matrix.npy", model.sim)

calculate_metrics(measures)